In [7]:
import numpy as np
from sklearn.model_selection import train_test_split, train_test_split
import pandas as pd

from utils.data import extract_eeg_data, segment_data
from load_data import load_kfold_data
from utils.labels import get_pss_labels, get_stai_labels
from utils.valid_recs import get_valid_recs
#from features import all_features, hjorth_features, differential_entropy, kymatio_wave_scattering
from classifiers import knn_classification, svm_classification

import utils.variables as v

In [8]:
data_type = 'raw'
label_type = 'stai'

#train_data, test_data, val_data, train_labels, test_labels, val_labels = load_data(data_type, label_type, epoched = True, binary = True)
train_data, test_data, train_labels, test_labels = load_kfold_data(data_type, label_type, epoched = False, binary = True)


---- Generating all recordings ----
All records generated

---- Filtering out invalid recordings ----
ERROR 1) Failed to read data for recording P006_S002_001
ERROR 1) Failed to read data for recording P006_S002_002
ERROR 1) Failed to read data for recording P028_S002_001
ERROR 1) Failed to read data for recording P028_S002_002

---- Returning valid recordings ----
['P001_S001_001', 'P001_S001_002', 'P001_S002_001', 'P001_S002_002', 'P002_S001_001', 'P002_S001_002', 'P002_S002_001', 'P002_S002_002', 'P003_S001_001', 'P003_S001_002', 'P003_S002_001', 'P003_S002_002', 'P004_S001_001', 'P004_S001_002', 'P004_S002_001', 'P004_S002_002', 'P005_S001_001', 'P005_S001_002', 'P005_S002_001', 'P005_S002_002', 'P006_S001_001', 'P006_S001_002', 'P007_S001_001', 'P007_S001_002', 'P007_S002_001', 'P007_S002_002', 'P008_S001_001', 'P008_S001_002', 'P008_S002_001', 'P008_S002_002', 'P009_S001_001', 'P009_S001_002', 'P009_S002_001', 'P009_S002_002', 'P010_S001_002', 'P010_S002_001', 'P010_S002_002', 'P

In [9]:
print(train_data.shape)
print(train_labels.shape)
#Reshape data
train_data = np.reshape(train_data, (train_data.shape[0]*train_data.shape[1], train_data.shape[2]))
train_labels = np.repeat(train_labels, repeats = 8, axis = 1).reshape(-1,1)
train_labels = train_labels.ravel()
print(train_data.shape)
print(train_labels.shape)

test_data = np.reshape(test_data, (test_data.shape[0]*test_data.shape[1],test_data.shape[2]))
test_labels = np.repeat(test_labels, repeats = 8, axis = 1).reshape(-1,1)
test_labels = test_labels.ravel()
print(test_data.shape)
print(test_labels.shape)

(60, 8, 75000)
(60, 1)
(480, 75000)
(480,)
(152, 75000)
(152,)


In [10]:
#x, x_test, y, test_labels = train_test_split(x_epochs, y_epochs, test_size=0.2, random_state=1)
#x_train, x_val, train_labels, val_labels = train_test_split(x, y, test_size=0.25, random_state=1)


# freq_bands = np.array([1, 4, 8, 12, 30, 50])
# features = freq_band_features(dataset, freq_bands)
# features = hjorth_features(dataset)
# features = entropy_features(dataset)
# features = fractal_features(dataset)

### KNN

In [11]:
print(f'{data_type} data with {label_type} labels')
knn_classification(train_data, test_data, train_labels, test_labels)

raw data with stai labels


/Users/AnneJoo/.local/share/virtualenvs/MASTER-eeg-stress-det-zSpBlwVN/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
180 fits failed out of a total of 9000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/AnneJoo/.local/share/virtualenvs/MASTER-eeg-stress-det-zSpBlwVN/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/AnneJoo/.local/share/virtualenvs/MASTER-eeg-stress-det-zSpBlwVN/lib/python3.10/site-packages/sklearn/neighbors/_classification.py", line 213, in fit
    self._validate_params()



Results:
Classification accuracy: 0.822368 

 Confusion matrix:
[[78 10]
 [17 47]]
Accuracy, Sensitivity, Specificyty:

[82.24 82.11 82.46]


### SVM

In [12]:
print(f'{data_type} data with {label_type} labels')
svm_classification(train_data, test_data, train_labels, test_labels)


raw data with stai labels

Results:
Classification accuracy: 0.875000 

 Confusion matrix:
[[81  7]
 [12 52]]
Accuracy, Sensitivity, Specificyty:

[87.5  87.1  88.14]
